# Homework 3: Linear Programming, Simplex and Duality

---
## Exercise 9.2.11

<img src="figures/homework-03/hw03-exercise-9.2.11.png" width="500" />

a) To answer this question, we need to use the definition of the canonical linear programming problem: 


<img src="figures/lecture-02/lec02-canonical-linear-prog.png" width="500"/>

So by definition the answer is True.

b) We know that a vector $\bar{x} \in \mathscr{F}$ is an optimal solution if $f(\bar{x}) = \max_{x \in \mathscr{F}}{f(x)}$.

So the answer is false because the vector $\bar{x}$ itself must be in the feasible set $\mathscr{F}$. It is possible for an $\bar{x} \not \in \mathscr{F}$ --- i.e., a non-feasible vector --- to yield the maximum value of $f$.

In the follow figure, we see that that $2x_1 + 3x_2 = 96$ is maximum value of the linear function $f$. The optimal solution is when $x_1 = 30$ and $x_2 = 11$ i.e., the green point. Notice that the red points on the level line also yield $f(x_1, x_2)=96$, but these points are not optimal since they are outside the feasible set. 

<img src="figures/homework-03/hw03-example-5.png" width="500" />

In [1]:
import linprog as lp

A = [[1, 0],
     [0, 1],
     [1, 2]]
b = [30, 20, 54]
c = [2, 3]

res = lp.maximise(A, b, c)
lp.pretty_print(res)

<IPython.core.display.Math object>

---
## Exercise 9.4.11

<img src="figures/homework-03/hw03-exercise-9.4.11.png" width="400" />

First, we need to convert the problem to its dual problem by putting the values in a ''matrix'' and perform transpose.

\begin{equation*}
\begin{bmatrix}
\begin{array}{ccccc|c}
x_1 & x_2 & x_3 &  b \\ \hline 
  1 &   1 &   3 &  4 \\ 
  2 &   1 &   2 &  5 \\ \hline 
 16 &  10 &  20 &  1 \\
\end{array}
\end{bmatrix}
\Longrightarrow^{Transpose}
\begin{bmatrix}
\begin{array}{ccccc|c}
y_1 & y_2 &   c \\ \hline 
  1 &   2 &  16 \\ 
  1 &   1 &  10 \\ 
  3 &   2 &  20 \\ \hline 
  4 &   5 &   1 \\ 
\end{array}
\end{bmatrix}
\end{equation*}

From the resulting, we can read off the values for the dual problem and solve it using the simplex method:

In [2]:
from simplex import SimplexSolver 
A = [[1, 2],
     [1, 1],
     [3, 2]]
b = [16, 10, 20]
c = [4, 5]
solver = SimplexSolver()
res = solver.run_simplex(A, b, c, prob='max', enable_msg=True, latex_path=None)

   x_1   x_2   s_1   s_2   s_3    b    
|   1     2     1     0     0    16   | s_1
|   1     1     0     1     0    10   | s_2
|   3     2     0     0     1    20   | s_3
|  -4    -5     0     0     0     0   |
Current solution: {'x_1': 0, 'x_2': 0, 's_1': Fraction(16, 1), 's_2': Fraction(10, 1), 's_3': Fraction(20, 1), 'z': 0}

   x_1   x_2   s_1   s_2   s_3    b    
|   1     2     1     0     0    16   | s_1
|   1     1     0     1     0    10   | s_2
|   3     2     0     0     1    20   | s_3
|  -4    -5     0     0     0     0   |

There are negative elements in the bottom row, so the current solution is not optimal. Thus, pivot to improve the current solution. The entering variable is x_2 and the departing variable is s_1.


Perform elementary row operations until the pivot is one and all other elements in the entering column are zero.

   x_1   x_2   s_1   s_2   s_3    b    
|  1/2    1    1/2    0     0     8   | x_2
|  1/2    0   -1/2    1     0     2   | s_2
|   2     0    

When the entries in the bottom row are non-negative then we can read off optimal solution of the primal problem at the bottom row.

The solution of the primal problem is:

$x_1 = 7/4, x_2 = 0, x_3 = 3/4$ and $M=43$

---
## Exercise 9.4.21

<img src="figures/homework-03/exercise-21.png" width="900" />



Since the matrix contains negative values, we add $k=3$ to shift the game.

\begin{equation*}
\begin{bmatrix}
\begin{array}{ccccc|c}
  7 &   4 &   1 \\ 
  4 &   6 &   3 \\
  2 &   3 &   7 \\
\end{array}
\end{bmatrix}
\end{equation*}

The linear programming problem is to maximise $y_1 + y_2 + y_3$ subject to:

\begin{align*}
7 y_1 + 4 y_2 +   y_3 & \leq 1 \\
4 y_1 + 6 y_2 + 3 y_3 & \leq 1 \\
2 y_1 + 3 y_2 + 7 y_3 & \leq 1 \\
\end{align*}

and $y_1 \leq 0, y_2 \leq 0, y_3 \leq 0$.


The following algorithm converts the problem into a linear programming problem and uses the simplex method to solve the problem.

In [3]:
import numpy as np
from simplex import SimplexSolver 

def arr_to_str(array):
    return '[{}]'.format(', '.join(str(x) for x in array))

def solve_matrix_game(A):
    A = np.array(A)
    
    # Ensure that all entries are greater than 0.
    # If some of the entries are equal to or less than zero,
    # then add a fixed value k to each entry. This will not change
    # the optimal strategy for the two players. It will only add
    # an amount k to the value of the game.
    min_entry = A.min()
    k = 0
    if min_entry <= 0:
        k = min_entry * -1 + 1
    A += k
    
    # b and c are always ones
    b = np.ones(A.shape[0], dtype=np.int)
    c = np.ones(A.shape[1], dtype=np.int)
    
    # Solve using the simplex method
    solver = SimplexSolver()
    res = solver.run_simplex(A, b, c, prob='max', enable_msg=False, latex_path=None)
    
    # Compute player C's strategy, y_hat
    y_bar = np.array([res[k] for k in res.keys() if k.startswith('x_')])
    _lambda = np.array(y_bar).sum()
    y_hat = y_bar / _lambda
    
    # Compute player R's strategy, x_hat
    margin_values = solver.get_marginal_values()
    x_bar = np.array(margin_values)
    # x_delta = x_bar.sum()
    x_hat = x_bar / _lambda

    # Compute the value of the game
    # Substract the amount k added to the payoff matrix
    v = 1/_lambda - k

    print("Row player's strategy: {}".format(arr_to_str(x_hat)))
    print("Column player's strategy: {}".format(arr_to_str(y_hat)))
    print('Value of the game: {}'.format(v))
    #return x_hat, y_hat, v


In [4]:
A = [[ 4,  1,  -2],
     [ 1,  3,   0],
     [-1,  0,   4]]
solve_matrix_game(A)

Row player's strategy: [11/35, 9/35, 3/7]
Column player's strategy: [2/5, 8/35, 13/35]
Value of the game: 38/35


The expected value of the portfolio is $38/35$ based on a payoff matrix for an investment of 100 USD. 

With 35,000 USD to invest, Bob plays this game 350 times. The expected payout is 380 USD. By the end of the year, the expected value of the portfolio has increased to 35,380.

Using the optimal game strategy, Bob should invest $11/35 \approx 31 \%$ in stocks, $9/35 \approx 26 \%$ in bonds and $3/7 \approx 43  \%$ in gold.